In [ ]:
import sys
sys.path.append("..")

import random
import math
from io import BytesIO
from pathlib import Path
from collections import OrderedDict
from typing import Optional, Callable, List, Tuple, Iterable, Generator, Union

import PIL.Image
import PIL.ImageDraw
import plotly
import plotly.express as px
plotly.io.templates.default = "plotly_dark"

import numpy as np
import pandas as pd
from tqdm import tqdm
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, TensorDataset, IterableDataset, RandomSampler
import torchvision.transforms as VT
import torchvision.transforms.functional as VF
from torchvision.utils import make_grid
import torchvision.models as VM 
from IPython.display import display

from src.util.image import *
from src.util import *
from src.models.util import *
from src.algo import ca1

def resize(img, scale: float, mode: VF.InterpolationMode = VF.InterpolationMode.NEAREST):
    return VF.resize(img, [max(1, int(s * scale)) for s in img.shape[-2:]], mode, antialias=False)

In [ ]:
if 0:
    vgg = VM.vgg19(weights=VM.VGG19_Weights.DEFAULT)
    print(f"params: {num_module_parameters(vgg):,}")
    vgg

In [ ]:
VF.to_pil_image(get_model_weight_images(vgg, normalize="each"))

In [ ]:
if 0:
    incept = VM.inception_v3(weights=VM.Inception_V3_Weights.DEFAULT)
    print(f"params: {num_module_parameters(incept):,}")
    incept

In [ ]:
if 1:
    shufflenet = VM.shufflenet_v2_x2_0(weights=VM.ShuffleNet_V2_X2_0_Weights.DEFAULT)
    print(f"params: {num_module_parameters(shufflenet):,}")    
    print(shufflenet)

# models

In [ ]:
for layer in vgg.features:
    if isinstance(layer, nn.Conv2d):
        print(layer)

In [ ]:
class VGGFeatures(nn.Module):
    def __init__(self, vgg: nn.Module):
        super().__init__()
        self.layers = vgg.features
        self.features = {}
        self._layer_map = {}
        for layer in self.layers:
            if isinstance(layer, nn.Conv2d):
                self._layer_map[layer] = f"conv_{len(self._layer_map) + 1}"
                self.features[self._layer_map[layer]] = None
                
    def forward(self, image):
        x = image
        for layer in vgg.features:
            x = layer(x)
            if layer in self._layer_map:
                self.features[self._layer_map[layer]] = x
        return x

    def features_concat(self, names: Optional[List[str]], gram: bool = True):
        features = []
        for name, f in self.features.items():
            if f is not None and (names is None or name in names):
                if gram:
                    f = f * f.permute(0, 1, 3, 2)
                features.append(f.flatten(-3))
        return torch.concat(features, dim=-1)
        

In [ ]:
class PixelModel(nn.Module):
    def __init__(self, image: torch.Tensor):
        super().__init__()
        self.image = nn.Parameter(image)

    def forward(self):
        return self.image
        

# training

In [ ]:
def train_image(
        source_image: str, 
        target_image: str,
        shape: Tuple[int, int] = (128, 128),
        epochs: int = 1000,
        device: str = "auto",
        batch_size: int = 16,
        lr: float = 0.01,
):
    device = to_torch_device(device)

    def _load_image(image: str):
        if isinstance(image, torch.Tensor):
            image = image.to(device)
        else:
            image = VF.to_tensor(PIL.Image.open(Path(image).expanduser()))[:3].to(device)
        
        if any(image.shape[-i] < shape[-i] for i in (1, 2)):
            image = image_resize_crop(image, shape)
        display(VF.to_pil_image(image))
        return image

    source_image = _load_image(source_image)
    target_image = _load_image(target_image)

    style_feature_names = [
        f"conv_{i}" for i in range(16, 17)
    ]
    
    model = VGGFeatures(vgg)
    model.to(device)
    for p in model.parameters():
        p.requires_grad = False
    print(f"params: {num_module_parameters(model):,}")
    print(model.features.keys())

    patches = torch.concat([p.unsqueeze(0) for p in iter_image_patches(target_image, shape=shape)])
    with torch.no_grad():
        model(patches)
        target_features = model.features_concat(names=style_feature_names).mean(dim=0)
        print(f"feature shape: {target_features.shape}")
        #print(f"feature shape: { {key: f.shape for key, f in target_features.items()} }")
        target_features = target_features.unsqueeze(0).expand(batch_size, *([-1] * target_features.ndim))
    
    pixel_model = PixelModel(source_image).to(device)
    target_image = target_image.to(device)
    
    optimizer = torch.optim.Adam(pixel_model.parameters(), lr=lr)
    scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer, .9999)

    transforms = VT.Compose((
        VT.RandomAffine(degrees=10),
        VT.RandomCrop(shape),
    ))
    
    history = {"loss": [], "lr": []}
    images = []
    try:
        for epoch in tqdm(range(epochs)):

            image_batch = torch.concat([transforms(pixel_model()).unsqueeze(0) for _ in range(batch_size)])
            #display(VF.to_pil_image(make_grid(image_batch))); return
            
            model(image_batch)
            features = model.features_concat(style_feature_names)

            loss = ((features - target_features) ** 2).sum() / 10_000
            #loss = F.mse_loss(features.mean(dim=0), target_features)
            
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            scheduler.step()

            with torch.no_grad():
                history["loss"].append(float(loss))
                lr = scheduler.get_last_lr()
                if isinstance(lr, (list, tuple)):
                    lr = lr[0]
                history["lr"].append(lr)
    
                if epoch % 20 == 0:
                    images.append(pixel_model().detach().cpu())
    
                if len(images) >= 10:
                    print(f"loss {float(loss)}")
                    display(VF.to_pil_image(make_grid(images, nrow=len(images))))
                    images.clear()
                
    except KeyboardInterrupt:
        pass

    with torch.no_grad():
        display(VF.to_pil_image(pixel_model().detach().cpu()))
        
    display(px.line(pd.DataFrame(history)))
    
train_image(
    #"~/Pictures/kali.png",
    torch.randn(3, *([128 * 4] * 2)) * .1 + .3,
    #"~/Pictures/matt.png",
    #"~/Pictures/kali.png",
    #"~/Pictures/DWlZbQ5WsAQEzHT.jpg",
    "~/Pictures/wang-template-3-edge.png",
    lr=0.01,
)